In [28]:
import numpy as np
import weakref  # 添加弱引用来避免循环引用
import contextlib  # 使用with 语句实现反向传播模式的控制


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x
# 只在有限范围内进行禁止反向传播


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        # 要求输入一个ndarray的数组
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.name = name
        self.generation = 0

    def __len__(self):
        return len(self.data)

    # 重载print函数
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n'+''*9) # ndarray转为str 对于换行加9个空格
        return 'variable(' + p + ')'

    def __mul__(self,other):
        return mul(self,other)


    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    # 当多次处理同一个变量，要重置导数
    def cleargrad(self):
        self.grad = None

    # 利用装饰器property使得shape等方法可以作为实例变量被访问，
    # 这里将ndarray的三个实例变量放入到variable类中

    @property
    def dtype(self):
        return self.data.dtype

    @property
    def size(self):
        return self.data.size

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def shape(self):
        return self.data.shape

    def backward(self, retain_grad=False):
        # 不用对最后的dy进行手动设grad为1
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        # 修改funcs的添加逻辑，处理复杂计算图的梯度优先问题
        # funcs = [self.creator]
        # 下面while只支持单个输入输出
        # while funcs:
        #     f = funcs.pop()
        #     x, y = f.input, f.output
        #     x.grad = f.backward(y.grad)
        #     if self.creator is not None:
        #         funcs.append(x.creator)
        funcs = []
        seen_set = set()
        # 调用add_func 函数来添加现在变量的creator seen_set是为了防止重复添加，funcs是为了排序来处理复杂计算图

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # 取出输出的梯度
            gxs = f.backward(*gys)  # 反向传播得到输入的梯度
            # 鉴定是否为元组，或者说数据保存为元组是因为会出现return x1, x2这种类型
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            # 使用zip来设置每一对的导数
            for x, gx in zip(f.inputs, gxs):
                # 这里是用输出端传播的导数进行赋值的，如果是两个一样的变量，那么没有相加而是赋值了两次
                # x.grad = gx
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                if x.creator is not None:
                    add_func(x.creator)
            # 上面代码是在反向传播中对输入的操作，这里是在进行方向传播时 每进行完一次传播
            # 就会将输出的梯度置0
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # 将中间变量的梯度内存删除


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y))for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])  # 设置辈分
            for output in outputs:  # 设置计算图（输出的creator）
                output.set_creator(self)

        # 训练过程需要反向传播求出导数，推理过程只进行正向传播，可以把中间过程扔掉
        self.inputs = inputs
        self.outputs = [weakref.ref(output)for output in outputs]
        #
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError

    def backward(self, gys):
        raise NotImplementedError


class Config:
    enable_backprop = True


class Mul(Function):
    def forward(self, x0,x1):
        y = x0*x1
        return y
    def backward(self, gy):
        x0 ,x1 = self.inputs[0].data,self.inputs[1].data
        return gy*x1,gy*x0
def mul(x0,x1):
    return Mul()(x0,x1)

In [29]:
class Add(Function):
    def forward(self, x0,x1):
        y = x1+x0
        return y
    def backward(self, gy):
        return gy, gy
    
def add(x0,x1):
    return Add()(x1,x0)

In [30]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2*x*gy
        return gx
def square(x):
    return Square()(x)

In [31]:
class Mul(Function):
    def forward(self, x0,x1):
        y = x0*x1
        return y
    def backward(self, gy):
        x0 ,x1 = self.inputs[0].data,self.inputs[1].data
        return gy*x1,gy*x0
def mul(x0,x1):
    return Mul()(x0,x1)

In [32]:
# 除了在variable类中定义特殊方法，也可以用下面的方法设置运算符重载
Variable.__mul__ =mul
Variable.__add__ =add

In [33]:
x0 = Variable(np.array(2.0))
x1 = Variable(np.array(3.0))
z = add(square(x0),square(x1))
z.backward()
print(z.data)
print(x0.grad)
print(x1.grad)

13.0
4.0
6.0


In [34]:
# 验证重复使用一个变量能不能处理
x = Variable(np.array(3.0))
y =  add(add(x,x),x)
y.backward()
print(x.grad)

3.0


In [35]:
x = Variable(np.array(2.0))
a = square(x)
y = add (square(a),square(a))
y.backward()
print(x.grad)
print(y.data)

64.0
32.0


In [ ]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))
y = a*b+c
y.backward()
print(y)
print(a.grad)
print(b.grad)

###  Cpython 使用两种方式管理内存: 引用计数 分代垃圾回收
#### 引用计数是对象创建时引用计数为0 当它被引用时 数+1 为0 删除
#### 当循环引用时 需要用分代垃圾回收
#### 我们实现的variable 和 Function 里有循环引用
![](../images/circlecite.png)
#### 添加完weakref后的计算图，可以看到如果在for循环中下一次计算时
#### 不会像上面一样释放不了中间变量（因为有outputs的引用）
![](../images/weakref.png)

In [36]:
# Python中的with用于自动进行后处理
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield # 预处理
    finally:
        print('done')  # 解释了后处理
with config_test():
    print('processing...')

start
processing...
done


In [37]:
# 需要临时将模式切换为禁用反向传播模式 例如在神经网络训练阶段 为了评估模型（训练阶段）常常使用不需要梯度的模式
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)